In [1]:
import csv
import datetime
import json

import ccxt
import pandas as pd
import pandas_ta as ta # needed for processing chart data 
import torch
import torch.nn as nn
from tqdm import tqdm
from transformers import AutoTokenizer, AlbertTokenizer, AutoModelForSequenceClassification, pipeline
from tokenization_roberta_spm import FairSeqRobertaSentencePieceTokenizer

In [2]:
tokenizer = AlbertTokenizer.from_pretrained("totoro4007/cryptobert-base-all-finetuned") 
model = AutoModelForSequenceClassification.from_pretrained("totoro4007/cryptobert-base-all-finetuned") 
model.resize_token_embeddings(len(tokenizer))

device = torch.device('cuda')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(52001, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [3]:
def classifier(title, content):
    encoded_inputs = tokenizer(title, content, return_tensors="pt", max_length=512, truncation=True).to(device)
    with torch.no_grad():
        output = model(**encoded_inputs)
        logits = output['logits']
    res = nn.Softmax(dim=1)(logits)[0]
    return res

def res2vec(res):
    vec = [0, 0, 0]
    for r in res:
        label = int(r['label'].split('_')[-1])
        score = round(r['score'], 4)
        vec[label] = score
    return vec

def vec_sum(vec1, vec2):
    return [vec1[0]+vec2[0], vec1[1]+vec2[1], vec1[2]+vec2[2]]

In [ ]:
news = {}
news_file="full_news_labeled.csv"
with open(news_file) as csvfile:
    reader = csv.reader(csvfile)
    for i, row in tqdm(enumerate(reader)):
        if i == 0:
            columns = row
            print(columns)
            continue
        date = f"{row[3]}-{row[4]}-{row[5]}-{row[6]}"
        text = f"{row[0]}. {row[1]}"
        
        res_vec = classifier(row[0], row[1])
        #res_vec = res2vec(res)
        
        mask = [0, 0, 0]
        mask[int(row[2])] = 1
        
        if date in news:
            news[date][0] += 1
            news[date][1] = vec_sum(news[date][1], res_vec)
            news[date][2] = vec_sum(news[date][2], mask)
        else:
            news[date] = [1, res_vec, mask]
        
        #print(date, news[date])
        #break

0it [00:00, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
5it [00:00, 46.20it/s]

['title', 'content', 'labels', 'year', 'month', 'day', 'hour']


30it [00:00, 46.21it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
86it [00:01, 47.41it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
91it [00:02, 46.97it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
112it [00:02, 48.77it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
455

4372it [01:15, 65.24it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
4470it [01:17, 59.67it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
4577it [01:19, 59.77it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
4591it [01:19, 61.86it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been remov

10260it [02:45, 68.18it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
10315it [02:46, 75.60it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
10396it [02:47, 75.45it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
10558it [02:49, 72

12569it [03:16, 77.74it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
12601it [03:17, 73.50it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
12737it [03:19, 76.37it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
12801it [03:19, 76.96it/s]

In [ ]:
def dates2datetimes(dates):
    res = []
    for date in dates:
        date = list(map(int, date.split('-')))
        neo = datetime.datetime(date[0], date[1], date[2], date[3])
        res.append(neo)
    return res

def closest(query, dates):
    dates = dates2datetimes(dates)
    st = 0
    en = len(dates)-1
    while True:
        mid = (st + en) // 2
        if dates[mid] < query:
            st = mid
        else:
            en = mid
        if mid == (st + en) // 2:
            break            
    return mid

def isin(query, date):
    date = list(map(int, date.split('-')))
    date = datetime.datetime(date[0], date[1], date[2], date[3])
    hour = datetime.timedelta(hours=1)
    delta = query - date
    delta = delta / hour
    if 0 < delta and delta <= 4:
        return True
    return False

In [ ]:
with open("BTC_USDT-4h_interval.json") as f: 
    d = json.load(f) 
    
chart_df = pd.DataFrame(d) 
chart_df = chart_df.rename(columns={0:"timestamp",
                                    1:"open",
                                    2:"high",
                                    3:"low", 
                                    4:"close",
                                    5:"volume"}) 

def process(df):
    binance = ccxt.binance() 
    dates = df['timestamp'].values 
    timestamp = [] 
    for i in range(len(dates)):
        date_string = binance.iso8601(int(dates[i])) 
        date_string = date_string[:10] + " " + date_string[11:-5] 
        timestamp.append(date_string) 
    df['datetime'] = timestamp 
    df = df.drop(columns={'timestamp'}) 
    return df

chart_df = process(chart_df) 

hours, days, months, years = [],[],[],[] 

for dt in tqdm(chart_df['datetime']):
    dtobj = pd.to_datetime(dt) 
    hour = dtobj.hour 
    day = dtobj.day 
    month = dtobj.month 
    year = dtobj.year 
    hours.append(hour)
    days.append(day) 
    months.append(month)
    years.append(year) 
    
chart_df['years'], chart_df['months'], chart_df['days'], chart_df['hours'] = years, months, days, hours
    
high_change, low_change = [], [] 
close = chart_df['close'].values 
high = chart_df['high'].values 
low = chart_df['low'].values 
for i in range(close.shape[0]-1):
    high_delta = (high[i+1] - close[i]) / close[i] 
    low_delta = (low[i+1] - close[i]) / close[i]
    high_change.append(high_delta) 
    low_change.append(low_delta)
high_change.append(None) 
low_change.append(None)

chart_df['high_delta'] = high_change
chart_df['low_delta'] = low_change 

chart_df['sent_0'] = 0.0
chart_df['sent_1'] = 0.0
chart_df['sent_2'] = 0.0

chart_df.dropna(inplace=True) 

chart_df

In [ ]:
chart_df.iloc[10199]

In [ ]:
news

In [ ]:
for index, row in tqdm(chart_df.iterrows()):
    query = datetime.datetime(row['years'], row['months'], row['days'], row['hours'])
    lk = list(news.keys())
    idx = closest(query, lk)
    ks = []
    n = 0
    for k in lk[max(0, idx-5):idx]:
        if isin(query, k):
            ks.append(k)
            n += news[k][0]
    for k in ks:
        try:
            v = news[k][1].cpu().detach().numpy()
        except:
            v = news[k][1]
            for i in range(3):
                v[i] = v[i].cpu().detach().numpy()
        chart_df.at[index, 'sent_0'] += v[0] / n
        chart_df.at[index, 'sent_1'] += v[1] / n
        chart_df.at[index, 'sent_2'] += v[2] / n
        
chart_df

In [ ]:
#print(chart_df.iloc[10199])
for index, row in tqdm(chart_df.iterrows()):
    if row.sent_0 > 0:
        print(row)
        break

In [ ]:
chart_df.set_index(pd.DatetimeIndex(chart_df['datetime']), inplace=True) 

chart_df['bop'] = chart_df.ta.bop(lookahead=False)
chart_df['ebsw'] = chart_df.ta.ebsw(lookahead=False) 
chart_df['cmf'] = chart_df.ta.cmf(lookahead=False) 
chart_df['rsi/100'] = chart_df.ta.rsi(lookahead=False) / 100 
chart_df['vwap'] = chart_df.ta.vwap(lookahead=False) 
chart_df['high/low'] = chart_df['high'] / chart_df['low'] 
chart_df['close/open'] = chart_df['close'] / chart_df['open'] 
chart_df['high/open'] = chart_df['high'] / chart_df['open'] 
chart_df['low/open'] = chart_df['low'] / chart_df['open'] 

chart_df['hwma'] = chart_df.ta.hwma(lookahead=False)
chart_df['linreg'] = chart_df.ta.linreg(lookahead=False)
chart_df['hwma/close'] = chart_df['hwma'] / chart_df['close'] 
chart_df['linreg/close'] = chart_df['linreg'] / chart_df['close']

for i in tqdm(range(1, 4)): 
    for col in ['open', 'high', 'low', 'close', 'volume', 'vwap']:
        val = chart_df[col].values 
        val_ret = [None for _ in range(i)] 
        for j in range(i, len(val)): 
            if val[j-i] == 0:
                ret = 1 
            else:
                ret = val[j] / val[j-i] 
            val_ret.append(ret) 
        chart_df['{}_change_{}'.format(col, i)] = val_ret 
            
    

chart_df.dropna(inplace=True) 
chart_df.drop(columns={'datetime', 'open', 'high', 'low', 'close', 'volume', 'vwap', 'hwma', 'linreg', 'years'}, inplace=True) 

In [ ]:
chart_df.head(2)

In [ ]:
chart_df.to_csv('bert.csv')